In [1]:

%load_ext sql

In [2]:
%sql postgresql://localhost/upreads_dev

In [7]:
%%sql
drop table users

 * postgresql://localhost/upreads_dev
Done.


[]

In [15]:
%%sql
drop table shelves

 * postgresql://localhost/upreads_dev
Done.


[]

In [14]:
%%sql
drop table user_shelves

 * postgresql://localhost/upreads_dev
Done.


[]

In [4]:
%%sql
drop table user_books

 * postgresql://localhost/upreads_dev
Done.


[]

In [3]:
%%sql
drop table shelf_books

 * postgresql://localhost/upreads_dev
Done.


[]

In [8]:
%%sql
CREATE TABLE IF NOT EXISTS users (
    id SERIAL PRIMARY KEY,
    username VARCHAR(255) UNIQUE NOT NULL,
    email VARCHAR(255) UNIQUE NOT NULL,
    password_digest TEXT NOT NULL
);

 * postgresql://localhost/upreads_dev
Done.


[]

In [16]:
%%sql 
CREATE TABLE IF NOT EXISTS shelves(
    id SERIAL PRIMARY KEY,
    is_public BOOLEAN,
    creator_user_id INTEGER,
    shelf_name VARCHAR(255)
);

 * postgresql://localhost/upreads_dev
Done.


[]

In [17]:
%%sql 
CREATE TABLE IF NOT EXISTS user_shelves(
    id SERIAL PRIMARY KEY,
    user_id INTEGER,
    shelf_id INTEGER REFERENCES shelves(id),
    UNIQUE (user_id, shelf_id)
);

 * postgresql://localhost/upreads_dev
Done.


[]

In [11]:
%%sql 
CREATE TABLE IF NOT EXISTS user_books(
    id SERIAL PRIMARY KEY,
    user_id INTEGER,
    google_book_id VARCHAR(255),
    status VARCHAR(255),
    shelf_id INTEGER,
    title VARCHAR(255),
    author VARCHAR(255),
    cover_img VARCHAR(1000),
    UNIQUE(user_id, shelf_id, google_book_id)
);

 * postgresql://localhost/upreads_dev
Done.


[]

In [12]:
%%sql
CREATE TABLE IF NOT EXISTS shelf_books(
    id SERIAL PRIMARY KEY,
    shelf_id INTEGER,
    google_book_id VARCHAR(255),
    title VARCHAR(255),
    author VARCHAR(255),
    cover_img VARCHAR(1000)
);   

 * postgresql://localhost/upreads_dev
Done.


[]

In [4]:
%%sql 
INSERT INTO shelves
    (is_public, creator_user_id, shelf_name)
VALUES
    (true, 1, 'Black Lives Matter'),
    (true, 2, 'Black Women Authors, Fiction')
;    

 * postgresql://localhost/upreads_dev
2 rows affected.


[]

In [5]:
%%sql
INSERT INTO user_shelves
    (user_id, shelf_id)
VALUES
    (1, 1),
    (2, 2)
;

 * postgresql://localhost/upreads_dev
2 rows affected.


[]

In [6]:
%%sql
INSERT INTO shelf_books
    (shelf_id, google_book_id, title, author, cover_img)
VALUES
    (1, 'REKIbk12HxkC', 'Point Blank', 'Anthony Horowitz', 'http://books.google.com/books/content?id=REKIbk12HxkC&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api'),
    (1, 'cYzzDwAAQBAJ', 'Alex Rider: Secret Weapon', 'Anthony Horowitz', 'http://books.google.com/books/content?id=cYzzDwAAQBAJ&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api' ),
    (2, 'wSSZDwAAQBAJ', 'Nightshade', 'Anthony Horowitz', 'http://books.google.com/books/content?id=wSSZDwAAQBAJ&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api')
;

 * postgresql://localhost/upreads_dev
3 rows affected.


[]

In [7]:
%%sql
INSERT INTO user_books
    (user_id, status, google_book_id, shelf_id, title, author, cover_img)
VALUES
    (1, 'unread', 'REKIbk12HxkC', 1, 'Point Blank', 'Anthony Horowitz', 'http://books.google.com/books/content?id=REKIbk12HxkC&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api'),
    (1, 'unread','cYzzDwAAQBAJ', 1, 'Alex Rider: Secret Weapon', 'Anthony Horowitz', 'http://books.google.com/books/content?id=cYzzDwAAQBAJ&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api' ),
    (2, 'unread', 'wSSZDwAAQBAJ', 2, 'Nightshade', 'Anthony Horowitz', 'http://books.google.com/books/content?id=wSSZDwAAQBAJ&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api')
    ;

 * postgresql://localhost/upreads_dev
3 rows affected.


[]

## Get user books
- get all books that a user has saved by shelf

In [16]:
%%sql
select * from user_books where user_id = 1;

 * postgresql://localhost/upreads_dev
2 rows affected.


id,user_id,google_book_id,status,shelf_id,title,author,cover_img
1,1,REKIbk12HxkC,unread,1,Point Blank,Anthony Horowitz,http://books.google.com/books/content?id=REKIbk12HxkC&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api
2,1,cYzzDwAAQBAJ,unread,1,Alex Rider: Secret Weapon,Anthony Horowitz,http://books.google.com/books/content?id=cYzzDwAAQBAJ&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api


In [4]:
%%sql
SELECT
    shelf_name,
    user_shelves.shelf_id,
    google_book_id
FROM
    (
        SELECT
            shelf_name,
            us.shelf_id
        FROM user_shelves us
        LEFT JOIN shelves on us.shelf_id = shelves.id
        WHERE us.user_id = 1
    ) user_shelves
LEFT JOIN shelf_books on user_shelves.shelf_id = shelf_books.shelf_id


 * postgresql://localhost/upreads_dev
5 rows affected.


shelf_name,shelf_id,google_book_id
Black Lives Matter,1,1BLM
Black Lives Matter,1,2BLM
Black Lives Matter,1,3BLM
"Black Women Authors, Fiction",2,1WOMEN
"Black Women Authors, Fiction",2,2WOMEN


## user shelf books with read status

In [5]:
%%sql
SELECT
    u_books.shelf_name,
    u_books.shelf_id,
    u_books.google_book_id,
    u_books.title,
    u_books.author,
    u_books.cover_img,
    status
FROM(
    SELECT
        shelf_name,
        user_shelves.shelf_id as shelf_id,
        google_book_id,
        title,
        author,
        cover_img
    FROM
        (
            SELECT
                shelf_name,
                us.shelf_id
            FROM user_shelves us
            LEFT JOIN shelves on us.shelf_id = shelves.id
            WHERE us.user_id = 1
        ) user_shelves
    LEFT JOIN shelf_books on user_shelves.shelf_id = shelf_books.shelf_id
    ) u_books
LEFT JOIN user_books on u_books.shelf_id = user_books.shelf_id
    AND u_books.google_book_id = user_books.google_book_id


 * postgresql://localhost/upreads_dev
12 rows affected.


shelf_name,shelf_id,google_book_id,title,author,cover_img,status
Black Feminist Philosophy,1,icWTAgAAQBAJ,Black Feminist Thought,Patricia Hill Collins,http://books.google.com/books/content?id=icWTAgAAQBAJ&printsec=frontcover&img=1&zoom=5&edge=curl&imgtk=AFLRE72AYQvAC83-wuxVSWFev2jsoACkAQ4rTgAwtyxD_VZuq-QWFa4Tz33pR5u3NlGdk6bp5itJVn4qp5mM9RcjFQxNbUF3vWscokL7E8BvKR4KI2dKJCnHFjH_6BwCszp4b4CDr0g3&source=gbs_api,unread
Black Feminist Philosophy,1,m0TJX5mc6gcC,In Search of Our Mothers Gardens,Alice Walker,http://books.google.com/books/content?id=m0TJX5mc6gcC&printsec=frontcover&img=1&zoom=5&edge=curl&imgtk=AFLRE700mQ4_pxsqXN98b2JtbcZZHjSpBaceGh_KcPssChC4ysPZhlZnw5h6JGeEyCkjp-AkobUMdX_83b8DVSdhzY2723ZOcZVZGVnx62-T92EroxtlLCWtXpWSBivsS_788lU5Evqq&source=gbs_api,unread
Black Feminist Philosophy,1,MpN0ikR6-f4C,Talking Back,Bell Hooks,http://books.google.com/books/content?id=MpN0ikR6-f4C&printsec=frontcover&img=1&zoom=5&edge=curl&imgtk=AFLRE73q_gVGvkQtcuOqGromv7vRrumiNhOLgjPB04bz2A4X4iPin1rZCl3XPSB_VgFBOXnWMWQ-hVzPMT3RIK6gKU4hWFQS2dtDU0KWzfDLqnEsgBVFm_WVRGYSwjzWVDXu4Ql0lMer&source=gbs_api,unread
Black Feminist Philosophy,1,_wZ35GI4itgC,Are Prisons Obsolete,Angela Y. Davis,http://books.google.com/books/content?id=_wZ35GI4itgC&printsec=frontcover&img=1&zoom=5&edge=curl&imgtk=AFLRE72xKbdbr27W3mXND4ubZ7szKnva4iPHvo66Ey9bM1ncWY68mP527tKfwhtXpqfTyxoyhEssEhZYYXQmrdPIttGucXUylZLLNB-F-fCvBu3k4zr4aOKxtlH4Edp7_IuMbzPhfEdQ&source=gbs_api,unread
Fiction by Women of Color,2,JTHJDwAAQBAJ,The Girl with the Louding Voice,Abi Daré,http://books.google.com/books/content?id=JTHJDwAAQBAJ&printsec=frontcover&img=1&zoom=5&edge=curl&imgtk=AFLRE711DR1MWVnGwNpP16GOgJTcHNMioxLUa-h2zP92apn-Wh7aMWu3vKRcJeUMg4-BXS5l0lbeO3QsoE9MFdt0zvCtk-9GUNBW8j46vp4mroRtuYB0nZo17Zz-etnrPjH6rjmjzyeY&source=gbs_api,unread
Fiction by Women of Color,2,elmSDwAAQBAJ,Such a Fun Age,Kiley Reid,http://books.google.com/books/content?id=elmSDwAAQBAJ&printsec=frontcover&img=1&zoom=5&edge=curl&imgtk=AFLRE71cOs1wWPhTR-0nbI0t4bQ-dFCofUH5kF8w4Us5SlHC1xH3DuBqavOYYGwSZxDxWltto2TptUBms247xc8BV9sN-FZGyHF9fVACw_7RDNK0ecDiaAoNvc5MEInUbC4RqV6Xg1BR&source=gbs_api,unread
Fiction by Women of Color,2,VzTXsgEACAAJ,Behold the Dreamers,Ombolo Mbue,http://books.google.com/books/content?id=VzTXsgEACAAJ&printsec=frontcover&img=1&zoom=5&imgtk=AFLRE70PK821YI_wmKVuG6KQ7OcKI0WwCeokLXJuw5vWQW1NCEZ_mvMW97IThxpt11ViJGZuRhVFMyUYQpsG8r0FcEORhUA_nZDafMZjsjhliNHdG7jZaGIV0cQyULQLbVPNMtYTxMmb&source=gbs_api,unread
Fiction by Women of Color,2,KQA9DwAAQBAJ,Fruit of the Drunken Tree,Ingrid Rojas Conteras,http://books.google.com/books/content?id=KQA9DwAAQBAJ&printsec=frontcover&img=1&zoom=5&edge=curl&imgtk=AFLRE70bdbtaE4L2XABjbasJ07EZjfQZYWtXFIo1ukDp-MgP1hkriyIwnHP304b5dKLzzU12Qe4aC12aYM-O7R3Gzg-ejaZAQmz-cDfVt-xR6DHFkr2XnqCfn379ryBuFfb4U41E1Jhn&source=gbs_api,unread
Fiction by Women of Color,2,sfmp6gjZGP8C,Beloved,Toni Morrison,http://books.google.com/books/content?id=sfmp6gjZGP8C&printsec=frontcover&img=1&zoom=5&edge=curl&imgtk=AFLRE71jjtky4x_iPiVPLEQYSGpO9jog9Nz8ut14kQGlRl45MuOaBU21FnBIJSYtGIe3dFr0Ksa0iEC9jUkKJrseEwuoBDmShVOIjUmC4BefIp5fQhrk8qbIOOo3n3Kx5uazBexzUjdp&source=gbs_api,unread
Ruth Bader Ginsburg,3,u5DyDwAAQBAJ,Ruth Bader Ginsburg,Jane Sherron de Hart,http://books.google.com/books/content?id=u5DyDwAAQBAJ&printsec=frontcover&img=1&zoom=5&edge=curl&imgtk=AFLRE72L1OtKcmqa7v78lIi3hihVhcZpsxNf5rfAs0dlPpyfoQ0TbBiXb4oM6AlvSXSIKmj-KZZmL8fkM1QEvvr02TO0_aOGBup3AZaRR9n_UsonSQMaL6pVhgdz4vcj_IbLFgFrp0kh&source=gbs_api,unread


In [7]:
%%sql
SELECT
    shelf_name,
    shelves.id,
    google_book_id
FROM shelves
LEFT JOIN
    shelf_books on shelves.id = shelf_books.shelf_id
WHERE is_public = true
;



 * postgresql://localhost/upreads_dev
5 rows affected.


shelf_name,id,google_book_id,is_public
Black Lives Matter,1,1BLM,True
Black Lives Matter,1,2BLM,True
Black Lives Matter,1,3BLM,True
"Black Women Authors, Fiction",2,1WOMEN,True
"Black Women Authors, Fiction",2,2WOMEN,True


In [8]:
%%sql
select * from user_shelves;

 * postgresql://localhost/upreads_dev
2 rows affected.


id,user_id,shelf_id
1,1,1
2,1,2


In [46]:
%%sql
select * from user_books;

 * postgresql://localhost/upreads_dev
3 rows affected.


id,user_id,google_book_id,status,shelf_id,title,author,cover_img
1,1,REKIbk12HxkC,unread,1,Point Blank,Anthony Horowitz,http://books.google.com/books/content?id=REKIbk12HxkC&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api
2,1,cYzzDwAAQBAJ,unread,1,Alex Rider: Secret Weapon,Anthony Horowitz,http://books.google.com/books/content?id=cYzzDwAAQBAJ&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api
3,2,wSSZDwAAQBAJ,unread,2,Nightshade,Anthony Horowitz,http://books.google.com/books/content?id=wSSZDwAAQBAJ&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api


In [45]:
%%sql
select * from user_shelves;


 * postgresql://localhost/upreads_dev
2 rows affected.


id,user_id,shelf_id
1,1,1
2,2,2


In [22]:
%%sql

select * from shelves WHERE creator_user_id = 1;

 * postgresql://localhost/upreads_dev
1 rows affected.


id,is_public,creator_user_id,shelf_name
1,True,1,Black Lives Matter


In [3]:
%%sql

SELECT
    shelf_name,
    us.shelf_id
FROM user_shelves us
LEFT JOIN shelves on us.shelf_id = shelves.id
WHERE us.user_id = 1

 * postgresql://localhost/upreads_dev
2 rows affected.


shelf_name,shelf_id
Black Lives Matter,1
test,6
